# Analyse over Wijn
- Stan Meyberg (TICT-AI-V2A-19)
- Roeland Oostdam (TICT-AI-V2A-19)
- Ruben Klinkenberg (TICT-AI-V2A-19)


## onderzoeksvragen
1. In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?  
2. In hoeverre speelt de prijs een rol in de beoordeling van de wijn?
3. In hoeverre kan op basis van de chemische kenmerken voorspelt worden of het een witte of rode wijn is?
4. Welke kernwoorden zijn typerend voor een hoog scorende wijn?  

Wij zullen deze onderzoeksvragen zo goed mogelijk proberen te beantwoorden.
Voor de beantwoording van deze vragen maken wij gebruik van het Data Science proces.  
Dit proces ziet er als volgt uit:
### Het data science proces
1. Data collection
2. Data processing
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication


Allereerst zullen we de benodigde libraries importeren.
Deze libraries zullen we gebruiken voor het analyseren en het visualiseren van de data.

In [ ]:
# data analysis libraries 
import numpy as np
import pandas as pd

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# libraries for machine learning algorithms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

# library for statistical functions
import scipy.stats as stats


## Data Collection
De eerste dataset die we gaan gebruiken is aan ons aangeleverd. De dataset is een csv bestand en heet: 'redwine.csv'. In de dataset staat informatie en chemische kenmerken van wijnen in Portugal. 

Allereerst beginnen we met het importeren van de dataset en kijken we of de dataset correct is geïmporteerd door de eerste 5 regels op te vragen.

In [ ]:
# import the data files
df_red_wine = pd.read_csv("data/redwine.csv", sep=";")
df_red_wine.head()

### externe dataset
Aangezien onze eerste dataset alleen maar kenmerken van rode wijn bevat zijn wij gaan zoeken naar een dataset met kenmerken van witte wijn.  
  
De dataset komt van de volgende website: https://archive.ics.uci.edu/ml/datasets/wine+quality  
  
Ook deze dataset gaan we importeren om hierna te controleren of hij goed geïmporteerd is.

In [ ]:
df_white_wine = pd.read_csv("data/winequality-white.csv", sep=";")
df_white_wine.head()

## Data Processing
Aangezien de bestanden met de data al in een csv bestand staan kunnen deze direct in een dataframe worden ingelezen.  
Nu is het nog aan ons om te beslissen of er nog kolomnamen zijn die aangepast moeten worden en welke kolommen we gaan droppen.  
In totaal heeft deze dataset 22 kolommen. Om uit te zoeken welke kolommen essentieel zijn voor het verdere proces gaan we nu eerste kijken hoe de kolommen heten.

In [ ]:
df_red_wine.columns

Om goed te kunnen begrijpen met welke data we hier mee te maken hebben is hieronder voor iedere kolom de betekenis van de data die erin staat gegeven.  

- **id**  
    Een uniek nummer voor iedere rij.
- **Country**  
    Het land waar de wijn vandaan komt.
- **Description**  
    De beschrijving van de wijn.
- **Designation**  
    De wijngaard waar tenminste 85% van de druiven vandaan komen.
- **Points**  
    De hoeveelheid punten die de wijn heeft gekregen van de proever.
- **Price**  
    De prijs van de wijn.
- **Province**  
    De provincie waar de wijn.
- **Taster_name**  
    De volledige naam van de proever.
- **Title**  
    De titel die normaliter op het wijnetiket staat.
- **Variety**  
    Het type druif dat gebruikt wordt.
- **Winery**  
    Het bedrijf waar de wijn geproduceerd is.
- **Fixed acidity**  
    Zuren zijn zeer belangrijke bestanddelen van wijn en voegen zeer veel toe aan de smaak. Hoe hoger de hoeveelheid zuren des te zuurder de wijn wordt.
- **Volatile acidity**  
    De hoeveelheid azijnzuur in de wijn. Kan leiden tot een azijnachtige smaak als het in te grote hoeveelheden aanwezig is.
- **Citric acid**  
    Een zuur die gebruikt kan worden als natuurlijk conserveermiddel. Citroenzuur kan bijdragen aan de frisheid en smaak van de wijn.
- **Residual sugar**  
    De hoeveelheid suiker die over is na de fermentatie van de wijn. De hoeveelheid suiker in de wijn geeft de type van de wijn aan (droog, halfdroog, zoet).
- **Chlorides**  
    De hoeveelheid zout in de wijn.
- **Free sulfur dioxide**  
    De hoeveelheid zwaveldioxide dat vrij in de wijn zit. (Wat dus niet gebonden is aan andere chemicaliën in de wijn).
    Het zwaveldioxide voorkomt oxidatie van de wijn en wordt als conserveringsmiddel in veel levensmiddelen gebruikt.
    Zwaveldioxide wat niet vrij is, en dus al gebonden is aan andere stoffen in de wijn, heeft geen antioxiderende werking meer.
- **Total sulfur dioxide**  
    De totale hoeveelheid zwaveldioxide dat in de wijn zit.  
    Een te hoge concentratie zwaveldioxide kan de smaak verpesten.
    Een te lage concentratie zwaveldioxide kan ervoor zorgen dat er teveel bacteriën in de wijn blijven zitten waardoor het gevaarlijk kan zijn om te drinken.
- **Density**  
    De dichtheid van de wijn. De dichtheid kan verminderen door toevoeging van meer alcohol.
- **pH**  
    Is een maat voor de zuurgraad van een waterige oplossing. De schaal gaat van 0 (zuur) tot 14 (basisch). De meeste wijnen zitten tussen de 2.8 en de 4.0.
- **Sulphates (sulfites)**  
    Een additief die bijdraagt aan het vermeerderen van het gas zwaveldioxide.
- **Alcohol**  
    Het percentage alcohol in de wijn.

Voor het uitzoeken van de betekenis van deze termen/kolommen zijn de volgende bronnen gebruikt:  
- https://www.gall.nl/ontdek/wijn/de-zuurgraad-van-wijn/
- https://winecompliancealliance.com/vinyard-designation-on-a-wine-label/
- https://wineserver.ucdavis.edu/industry-info/enology/methods-and-techniques/common-chemical-reagents/citric-acid
- https://waterhouse.ucdavis.edu/whats-in-wine/volatile-acidity
- http://rstudio-pubs-static.s3.amazonaws.com/80458_5000e31f84df449099a872ccf40747b7.html


Bij deze dataset hebben we al de voorkennis dat al deze wijnen uit Portugal komen. Hierdoor hoeven we de kolom 'country' niet te gebruiken. Verder geeft 'variety' aan met wat voor druif we te maken hebben, waarschijnlijk zal dit ook maar één unieke waarde hebben.  
Voordat we deze kolommen weggooien checken we nog voor de zekerheid of onze aannames kloppen. 

In [ ]:
df_red_wine['country'].unique()

In [ ]:
df_red_wine['variety'].unique()

Onze aannames blijken te kloppen. We hebben hier twee keer te maken met een kolom met maar één unieke waarde. Deze kolommen kunnen dus gedropt worden.

In [ ]:
df_red_wine.drop(['country', 'variety'], axis=1, inplace=True)

Op basis van onze onderzoeksvragen kunnen we ook de kolommen 'Designation', 'winery' en 'Province' laten vallen. Dit omdat we voor de beantwoording van onze onderzoeksvragen geen onderscheid hoeven te maken tussen de verschillende provincies of de wijngaarden waar de wijn vandaan kan komen.

In [ ]:
df_red_wine.drop(['designation', 'province', 'winery'], axis=1, inplace=True)

Verder maken we van de kolom 'id' de index

In [ ]:
df_red_wine.set_index('id', inplace=True)
df_red_wine.head()

## Data cleaning
Nu de data bewerkt is kunnen we tot de volgende stap overgaan: het opschonen van de data. Hierbij gaan we op zoek naar missende en dubbele waarden, outliers en onvolkomenheden.  
Allereerst gaan we opzoek naar rijen waar waarden missen.

In [ ]:
len(df_red_wine)

In [ ]:
pd.isnull(df_red_wine).sum()

Hier zien we dat er 269 missende waarden zijn en dat die waarden zich allemaal in de 'price' kolom bevinden. Deze waarde hebben we echter wel voor één van de onderzoeksvragen. Hier hebben wij de afweging gemaakt om deze waarden te verwijderen.  
Deze keuze is gebaseerd op het feit dat we nu ongeveer 12% van de data weggooien en we dus nog genoeg data behouden om te onderzoeken of er een relatie zit tussen de prijs en de hoeveelheid punten van een fles wijn.  
We droppen dus alle rijen met missende waardes en tegelijkertijd verwijderen we duplicate rijen.  
  
Aangezien we hiermee ook extra informatie over chemische kenmerken weggooien hebben we de afweging gemaakt om hier twee verschillende dataframes aan te maken: één met de verwijderde rijen van 'price' en één zonder, zodat we voor de andere onderzoeksvragen alle kenmerken beschikbaar hebben die we kunnen krijgen. Voor de dataframe waarvan we de prijzen niet verwijderen worden deze prijzen omgezet tot de waarde 0.0

In [ ]:
# copying the dataframe
df_red_wine_dropped_price_rows = df_red_wine.copy()

# deleting the rows with empty values for prizes
df_red_wine_dropped_price_rows.dropna(inplace=True)

# filling the rows with empty values for prizes with 0.0
df_red_wine.fillna(0.0, inplace=True)

# deleting the duplicate rows
df_red_wine.drop_duplicates(inplace=True)
df_red_wine_dropped_price_rows.drop_duplicates(inplace=True)

Nu moeten we nog checken of elke kolom de geschikte datatype heeft.

In [ ]:
df_red_wine.dtypes

In de bovenstaande tabel valt te zien dat de kolom 'density', 'citric acid' en 'alcohol' niet de gewenste datatypen hebben. We willen hier floats hebben terwijl ze nu aangegeven worden als Strings.  
Om dit op te lossen gaan we over de kolommen heen en zetten iedere String om tot een float. Wanneer dit niet kan omdat de waarde dit niet toelaat wordt er een NaN ingevuld.  
  
De reden dat we deze stappen niet doen voor de 'red_wine_price_dropped' dataframe is dat we deze dataframe eigenlijk alleen maar gebruiken om te kijken of er een correlatie bestaat tussen twee kolommen. Hierbij komen geen van de chemische kenmerken kijken.

In [ ]:
df_red_wine['citric acid'] = pd.to_numeric(df_red_wine['citric acid'], errors='coerce')
df_red_wine['density'] = pd.to_numeric(df_red_wine['density'], errors='coerce')
df_red_wine['alcohol'] = pd.to_numeric(df_red_wine['alcohol'], errors='coerce')

pd.isnull(df_red_wine).sum()

In de bovenstaande tabel valt te lezen dat in totaal 194 waarden zijn omgezet tot NaN. Deze waardes kunnen we nu automatisch vullen door te interpoleren. Deze keuze hebben we gemaakt omdat we op zoek zijn naar bepaalde kenmerken en door te interpoleren blijven deze unieke kenmerken tussen de waardes die ze uniek maakt.

In [ ]:
df_red_wine.interpolate(inplace=True)
pd.isnull(df_red_wine).sum()

## Data Exploration

### De wijnproevers
Aangezien we met wijnproevers te maken hebben is alle data over de punten die aan een wijn toegedeeld zijn subjectief. Allereerst willen we dus wat inzicht krijgen over het gedrag van deze groep bij de toekenning van de punten.  
  
Eerst gaan we kijken met hoeveel wijnproevers we in totaal te maken hebben.

In [ ]:
len(df_red_wine['taster_name'].unique())

We hebben dus met 18 verschillende wijnproevers te maken.  
Hierna gaan we kijken of de hoeveelheid gegeven beoordelingen van de wijnproevers gelijk verdeeld zijn.

In [ ]:
df_red_wine['taster_name'].value_counts()

In de bovenstaande tabel valt te zien dat deze verdeling niet echt gelijk verdeeld is.  
Om deze data nog iets overzichtelijker te maken zetten we deze tabel om tot een grafiek.

In [ ]:
# selecting the data
df_tasting_freq = df_red_wine['taster_name'].value_counts()

# parsing the data
fig, ax = plt.subplots(figsize=(10,4), dpi=150)
ax.bar(df_tasting_freq.index, df_tasting_freq, width= 0.8)

# setting the axis
plt.xticks(rotation=90)
plt.yticks([n for n in range(0, 700, 50)])

# setting the style, labels and title
plt.xlabel('Naam proever')
plt.ylabel('Hoeveelheid beoordelingen')
plt.title('Hoeveelheid geregistreerde beoordelingen per proever')
sns.set_style("whitegrid")

# printing the plot
plt.show()

De aannames die de tabel aanwakkeren worden door de grafiek bevestigd. Uit de grafiek valt nog beter te zien dat de hoeveelheid beoordelingen per proever zeer onevenredig verdeeld zijn. Sommige proevers hebben zeer veel beoordelingen gegeven terwijl andere proevers veel minder hebben gegeven.  
  
Hierna willen we graag weten wat de minimale, maximale en gemiddelde score is per proever.

In [ ]:
# prepare the data_frame
df_taster_index = df_red_wine.copy()
df_taster_index = df_taster_index[['taster_name', 'points']]
df_taster_index.set_index('taster_name', inplace=True)

# getting the min, max and average per taster
taster_avg = df_taster_index.groupby('taster_name').mean()
taster_min = np.min(df_taster_index.groupby('taster_name'))
taster_max = np.max(df_taster_index.groupby('taster_name'))

# building a new dataframe with all the info per taster
df_taster_stats = pd.concat([taster_min, taster_max, taster_avg], axis=1)
df_taster_stats.columns = ['min points', 'max points', 'avg points']
df_taster_stats.sort_values(['avg points'])

In [ ]:
# print the total mean score over all tasters
print("Total Mean: {0}".format(df_red_wine['points'].mean()))

Uit deze tabel valt te lezen dat het gemiddeld aantal punten ongeveer rond de 88 punten zal liggen. Om nog iets meer duidelijkheid te creeëren rondom de puntenverdeling zullen we de gegeven punten in een grafiek plotten.

In [ ]:
# selecting the data
data = df_red_wine['points'].value_counts().sort_index()

# parsing the data
fix, ax = plt.subplots(figsize=(10,4), dpi=150)
ax.bar(data.index, data)

# setting the axis
plt.xticks([n for n in range(80, 101, 1)])

# setting the style, labels and title
plt.xlabel('Puntenaantal')
plt.ylabel('Hoeveelheid stemmen')
plt.title('Verdeling van de hoeveelheid stemmen per puntenaantal')
sns.set_style("whitegrid")

# printing the plot
plt.show()


Zoals in de grafiek te zien is valt zal het gemiddelde ongeveer rond de 88 liggen. Verder is de modus ook 88. Het interessante van deze grafiek is het feit dat zo rond 89 punten een kleine dip is. Dit kan ook mede komen door de aangeleverde data.

### Prijs analyse
Om wat meer informatie te vergaren over de prijzen is het bevorderlijk om de verdeling van de prijzen te plotten.

In [ ]:
data = df_red_wine[['points','price']]
data = data.groupby('points').mean()

fix, ax = plt.subplots(figsize=(12,4), dpi=150)

ax.bar(data.index, data['price'])
plt.xticks([n for n in range(81, 101, 1)])
plt.title("Gemiddelde prijs per punten")
plt.ylabel('Prijs');
plt.xlabel('Punten');
plt.show()

### Chemische kenmerken
Ook qua chemische kenmerken hebben willen we graag wat analyses doen over de dataset. Deze analyses kunnen we dan hopelijk gebruiken bij het selecteren van de variabelen om de twee soorten wijnen te klassificeren.  
  
Allereerst vragen we de algemene statistieken op over alle chemische kenmerken.

In [ ]:
df_red_wine.describe()

Op het eerste gezicht zeggen deze waardes nog niet zoveel. Pas wanneer we de kolommen met de minimale, maximale en gemiddelde waarden van alle kenmerken met van de witte en rode wijn zullen (hopelijk) deze waardes veel meer zeggen. 
  
Dus gaan we nu voor alle chemische kenmerken de minimale, maximale en gemiddelde waardes van witte en rode wijn met elkaar vergelijken.

## onderzoeksvraag 1

Hieronder zullen we proberen de volgende onderzoeksvraag uit te werken:  
#### In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?  
Wat hiermee bedoeld wordt is dat gegeven een bepaalde Portugese Red met bekende chemische kenmerken, of deze wijn op basis van zijn chemische kenmerken ingedeeld kan worden op een bepaalde score.

Allereerst zullen we hiervoor een basis voorspelling maken. In onze eerdere analyse kwam naar voren dat de modus van alle punten op 88 punten uitkwam. Op basis van deze informatie zullen we nu de eerste voorspellingen gaan doen. Hierbij voorspellen we dat iedere Portugese Red een score heeft van 88. Voor deze voorspelling willen we graag weten hoe accuraat hij is.

In [ ]:
df_modus_pred = pd.Series(88, index=[df_red_wine.index])
accuracy_score(df_red_wine['points'], df_modus_pred)

Ons eerste model geeft een accurate voorspelling van de score in 14% van de gevallen. Echter maken we hier geen onderscheid tussen de verschillende waarden die de chemische kenmerken kunnen hebben. Ofwel de chemische kenmerken worden bij deze voorspellingen buiten beschouwing gelaten.  
Om uiteindelijk de onderzoeksvraag zo volledig mogelijk te kunnen beantwoorden moeten we de chemische kenmerken meenemen in het proces.  
  
Om uit te zoeken of er tussen de gegeven scores en de chemische kenmerken überhaupt iets van correlatie te vinden is vragen we op alle basis van de gegeven punten de correlatie met de chemische kenmerken op.

In [ ]:
df_red_wine.corr()['points']

In de bovenstaande tabel valt te zien dat er qua correlatie tussen punten en chemische kenmerken geen sprake is van significante correlatie. Wel valt hier te zien dat er een mogelijke correlatie zou kunnen bestaan tussen de punten en de hoeveelheid alcohol die in de wijn zit. Desalniettemin zijn we wel benieuwd of we met behulp van lineaire regressie wel een voorspelling kunnen geven.  
We willen uiteindelijk namelijk een formule die 'voorspelt' wat de score zal zijn bij een bepaalde mix van chemische kenmerken.  
  
Allereerst wordt maken we een lineair regressie model aan. Hierna selecteren we de features en de uitkomst om deze om deze op te delen in een training en testset. Als laatste trainen we het model met de hiervoor aangemaakte trainingset.

In [ ]:
# Create the linear regression object
wine_regr_model = LinearRegression()

# Select the features and target
X = df_red_wine.loc[:,'fixed acidity':]
y = df_red_wine['points']

# create a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print("Aantal waarden in de training set: {0}".format(len(X_train)))
print("Aantal waardes in de test set: {0}".format(len(X_test)))

# Start the learning process
wine_regr_model.fit(X_train, y_train)

Nu het model getrained is zijn we geïntereseerd welke coëfficienten iedere feature heeft gekregen.

In [ ]:
pd.DataFrame({'features': X.columns, 'coefficients': wine_regr_model.coef_})

Hier zien we dat chorides heel veel invloed heeft. Verder hebben volatile acicity, sulphates en alcohol ook invloed op de lijn.  
Om uit te vinden hoe goed de voorspellende waarde van deze regressie lijn is gaan we hem voorspellingen laten maken met behulp van de test set. Hierna kunnen we door middel van de echte uitkomst en de 'voorspelde' uitkomst uitrekenen hoe accuraat de voorspellingen zijn. 

In [ ]:
# Make predictions using the test set
wine_regr_test_orig = wine_regr_model.predict(X_test)
wine_regr_test_rounded = np.rint(np.floor(wine_regr_model.predict(X_test)))

# Calculate the average deviation
avg_dev = abs(wine_regr_test_orig - y_test).sum() / len(y_test)

In [ ]:
# Calculate the root mean squared error
print("mean_squared_error: {0}".format(mean_squared_error(y_test, wine_regr_test_orig)))
print("Determinatiecoëfficient: {0}".format(wine_regr_model.score(X_test, y_test)))
print("Algemene score (met afronding): {0}".format(accuracy_score(wine_regr_test_rounded, y_test)))

print("\nGemiddelde afwijking: {0}".format(avg_dev))

Allereerst hebben we de mean_squared_error opgevraagd, deze score geeft aan hoe dicht de regressie lijn bij alle punten ligt. De bedoeling is om dit cijfer zo klein mogelijk te krijgen, aangezien dan de 'voorspellingen' accurater zijn.  
Hierna hebben we de determinatiecoëfficient, deze score geeft de variabiliteit van de waarden aan. Ofwel de water waarin verband lijkt te zijn op een rechte lijn. 0.29 is echter een niet al te hoge score voor een lineaire regressie.

De algemene score is berekend door alle voorspelde waardes naar beneden af te ronden tot een geheel getal. Dit geeft aan dat de voorspelling voor 17% van de waardes in de test set correct is.
Aangezien we ook graag willen weten in hoeverre de voorspellingen gemiddeld van de correcte voorspelling afliggen hebben we de gemiddelde afwijking berekend. Deze pakt het absolute verschil tussen de onafgeronde voorspellingen en de correcte waardes en telt deze bij elkaar op om uiteindelijk te delen door de totale hoeveelheid waardes in de test set. 
  
  
We weten dat alle scores in principe subjectief zijn toebedeeld. Dit zou één van de redenen kunnen zijn waarom hier niet of nauwelijks sprake is van een correcte lineaire regressie. Om hier wat meer inzicht in te krijgen bekijken we de correlatie tussen de gegeven punten van één bepaalde proever en de chemische eigenschappen.

In [ ]:
df_red_wine[df_red_wine['taster_name'] == 'Roger Voss'].corr()['points']

Zoals in de bovenstaande tabel te zien valt is er niet echt een groot verschil tussen de correlaties van één persoon en de correlaties van de gehele set van wijnproevers. desalniettemin hoeft dat niet te betekenen dat dit ook voor de rest van de wijnproevers het geval is. Wel valt er namelijk te zien dat de correlatie iets sterker is op sommige plekken.  
  


In [ ]:
# Create the linear regression object
selected_features_regr_model = LinearRegression()

# Select the features and target
#X = df_red_wine[['citric acid', 'sulphates', 'alcohol', 'volatile acidity', 'chlorides']]
X = df_red_wine[['alcohol', 'sulphates', 'chlorides']]
y = df_red_wine['points']

# create a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Start the learning process
selected_features_regr_model.fit(X_train, y_train)

# Make predictions using the test set
wine_regr_test_orig = selected_features_regr_model.predict(X_test)
wine_regr_test_rounded = np.rint(np.floor(selected_features_regr_model.predict(X_test)))

# Calculate the average deviation
avg_dev = abs(wine_regr_test_orig - y_test).sum() / len(y_test)

# Calculate the root mean squared error
print("mean_squared_error: {0}".format(mean_squared_error(y_test, wine_regr_test_orig)))
print("Determinatiecoëfficient: {0}".format(selected_features_regr_model.score(X_test, y_test)))
print("Algemene score (met afronding): {0}".format(accuracy_score(wine_regr_test_rounded, y_test)))

print("\nGemiddelde afwijking: {0}".format(avg_dev))

In [ ]:
def tune_decision_tree(depth):
    # Create the decision tree object
    wine_decision_tree = DecisionTreeClassifier(max_depth=depth)

    # Select the features and target
    X = df_red_wine.loc[:,'fixed acidity':]
    y = df_red_wine['points']

    # Split the data into a train set and analysis set
    X_train, X_analysis, y_train, y_analysis = train_test_split(X, y, random_state=0)

    # Create a validation and test set
    X_validate, X_test, y_validate, y_test = train_test_split(X_analysis, y_analysis, random_state=0, test_size=0.5)

    # fit the data
    wine_decision_tree.fit(X_train, y_train)

    # validation accuracy score
    val_score = wine_decision_tree.score(X_validate, y_validate)
    
    # test accuracy score
    test_score = wine_decision_tree.score(X_test, y_test)
    
    return (wine_decision_tree, val_score, test_score)

for x in range(1, 13):
    tree_result = tune_decision_tree(x)
    print("n: {0}; accuracy validation: {1}; accuracy test: {2}".format(x, tree_result[1], tree_result[2]))

In [ ]:
from sklearn.tree import export_graphviz

# Building the tree
decision_tree = tune_decision_tree(6)[0]

# Selecting the feature and target names
wine_feature_names = df_red_wine.loc[:,'fixed acidity':].columns
wine_target_names = df_red_wine['points'].unique()
wine_target_names

# Export as dot file
export_graphviz(decision_tree,
                feature_names = wine_feature_names,
                out_file='tree.dot',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')


Alcohol is de grootste discriminator

### Conclusie
Geen significante correlatie tussen de hoeveelheid punten die een bepaalde wijn heeft gekregen en de chemische kenmerken van de desbetreffende wijn. Hierdoor is het voorspellen door middel van lineaire regressie niet effectief wat ook blijkt uit de testen die we met dit algoritme hebben gedaan.  
De voorspellingen vanuit een decision tree waren iets effectiever

- Alle scores die aan een bepaalde wijn zijn gegeven zijn subjectief van aard.

## onderzoeksvraag 2
Hieronder zullen we proberen de volgende onderzoeksvraag uit te werken:  
#### In hoeverre speelt de prijs een rol in de beoordeling van de wijn?

## onderzoeksvraag 3

## onderzoeksvraag 4

## Hypothesetoets  
In het afgelopen jaar was het bijzonder zonnig in Portugal. De verbouwers van wijn vragen zich af of dit de zuurgraad (pH) van heeft beïnvloed, met andere woorden: of de wijnen uit het afgelopen jaar een andere pH hebben dan de wijnen uit andere jaren. Ze doen daarom een steekproef en meten de zuurgraad in enkele rode wijnen van afgelopen jaar. Daar komen de volgende meetwaarden uit:

In [ ]:
steekproef = np.array([3.41, 3.51, 3.39, 3.11, 3.21, 3.50, 3.46, 3.37, 3.71])

De onderzoeksvraag is: Hebben deze wijnen een significant andere zuurgraad dan gemiddeld?  
Om zometeen een conclusie te kunnen trekken moeten we eerst de hypotheses opstellen:  
- H0: Er is niet een verschil tussen de gemiddeldes van de wijnen uit de steekproef en de wijnen uit de totale populatie.
- H1: Er is een verschil tussen de gemiddeldes van de wijnen uit de steekproef en de wijnen uit de totale populatie.  
  
Er is sprake van een tweezijdige toets.

Allereerst berekenen we wat statistieken van zowel de gehele populatie als de steekproef.

In [ ]:
# gemiddelde populatie
mu = df_red_wine['pH'].mean()
# standaarddeviatie populatie
sigma = float(df_red_wine['pH'].std())

# grootte steekproef
n_sp = len(steekproef)
# gemiddelde steekproef
mean_sp = steekproef.mean()

print("Gemiddelde populatie: {0}\nStandaarddeviatie populatie: {1}".format(mu, sigma))
print("\nGrootte steekproef: {0}\nGemiddelde steekproef: {1}".format(n_sp, mean_sp))

Hierna bepalen we de foutmarge  
Gegeven: $\alpha = 0.05$

In [ ]:
alpha = 0.025

# Calculate the z-grens
neg_z_grens = stats.norm.ppf(alpha)
pos_z_grens = stats.norm.ppf(1 - alpha)

Nu willen we de grenswaarde van het kritieke gebied berekenen. Aangezien er te maken is met een tweezijdige toets hebben we nu twee grenswaardes nodig.
Dit doen we met behulp van de volgende formule:  
$gr = \mu + \frac{z \cdot \sigma}{\sqrt{n}}$

In [ ]:
pos_grens = mu + ((pos_z_grens * sigma) / n_sp**0.5)
neg_grens = mu + ((neg_z_grens * sigma) / n_sp**0.5)

print("Positieve grenswaarde: {0}\nNegatieve grenswaarde: {1}".format(pos_grens, neg_grens))

Het kritieke gebied is $x < 3.211$ en $x > 3.412$. Het gemiddelde van de steekproef is $3.408$ dus de waarde ligt niet in het kritieke gebied dus we hebben niet genoeg bewijs om de nulhypothese te kunnen verwerpen. Ofwel deze wijnen hebben niet een andere significante zuurgraad dan gemiddeld.